## 3日目

### 型を司るもの、カインド

In [3]:
import scalaz._, Scalaz._

import scalaz._, Scalaz._

In [1]:
:k Int

: 

* そうだろうと思ったけど、:kは使えないね。大人しくコンソール開くね

### Tagged type

In [2]:
sealed trait KiloGram

defined trait KiloGram

In [5]:
def KiloGram[A](a: A): A @@ KiloGram = Tag[A, KiloGram](a)

defined function KiloGram

In [6]:
val mass = KiloGram(20.0)

mass: Double @@ KiloGram = 20.0

In [7]:
2 * Tag.unwrap(mass)

res6: Double = 40.0

In [8]:
2 * scalaz.Tag.unsubst[Double, Id, KiloGram](mass)

res7: Double = 40.0

* これ意味のある値に名前をつけたいパターンあるある
* でも `Tag.unwrap()` って書くのは面倒じゃない? もっといい方法ないのかな

In [9]:
sealed trait JoulePerKiloGram

def JoulePerKiloGram[A](a: A): A @@ JoulePerKiloGram = Tag[A, JoulePerKiloGram](a)

defined trait JoulePerKiloGram
defined function JoulePerKiloGram

In [10]:
def energyR(m: Double @@ KiloGram): Double @@ JoulePerKiloGram = 
    JoulePerKiloGram(299792458.0 * 299792458.0 * Tag.unwrap(m))

defined function energyR

In [11]:
energyR(mass)

res10: Double @@ JoulePerKiloGram = 1.79751035747363533E18

In [11]:
energyR(10.0)

cmd11.sc:1: type mismatch;
 found   : Double(10.0)
 required: scalaz.@@[Double,cmd11Wrapper.this.cmd9.cmd1.wrapper.KiloGram]
    (which expands to)  AnyRef{type Tag = cmd11Wrapper.this.cmd9.cmd1.wrapper.KiloGram; type Self = Double}
val res11 = energyR(10.0)
                    ^

: 

### Monoidについて

In [12]:
4 * 1

res11: Int = 4

In [13]:
1 * 9

res12: Int = 9

In [14]:
List(1, 2, 3) ++ Nil

res13: List[Int] = List(1, 2, 3)

In [15]:
Nil ++ List(0.5, 2.5)

res14: List[Double] = List(0.5, 2.5)

In [16]:
(3 * 2) * (8 * 5) assert_=== 3 * (2 * (8 * 5))

In [17]:
List("la") ++ (List("di") ++ List("da")) assert_=== (List("la") ++ List("di")) ++ List("da")

* `*` や `++` は結合的(associativity) という
* `-` は結合的でない

### Monoid
* モノイドは、結合的な二項演算子(2引数関数) と、その演算に関する単位元からなる構造

```scala
trait Monoid[A] extends Semigroup[A] { self =>
  ////
  /** The identity element for `append`. */
  def zero: A
  
  ...
}
```

### Semigroup
* Monoid は Semigroup を継承している

```scala
trait Semigroup[A] { self =>
  def append(a1: A, a2: => A): A
  ...
}

trait SemigroupOps[A] extends Ops[A] {
  final def |+|(other: => A): A = A.append(self, other)
  final def mappend(other: => A): A = A.append(self, other)
  final def ÷(other: => A): A = A.append(self, other)
}
```

In [18]:
List(1, 2, 3) mappend List(4, 5, 6)

res17: List[Int] = List(1, 2, 3, 4, 5, 6)

In [19]:
"one" mappend "two"

res18: String = "onetwo"

In [20]:
List(1, 2, 3) |+| List(4, 5, 6)

res19: List[Int] = List(1, 2, 3, 4, 5, 6)

In [21]:
"one" |+| "two"

res20: String = "onetwo"

In [22]:
Monoid[List[Int]].zero

res21: List[Int] = List()

In [23]:
Monoid[String].zero

res22: String = ""

In [24]:
Tags.Multiplication(10) |+| Monoid[Int @@ Tags.Multiplication].zero

res23: Int @@ Tags.Multiplication = 10

In [25]:
10 |+| Monoid[Int].zero

res24: Int = 10

In [26]:
Tags.Disjunction(true) |+| Tags.Disjunction(false)

res25: Boolean @@ Tags.Disjunction = true

In [27]:
Monoid[Boolean @@ Tags.Disjunction].zero |+| Tags.Disjunction(true)

res26: Boolean @@ Tags.Disjunction = true

In [28]:
Monoid[Boolean @@ Tags.Disjunction].zero |+| Monoid[Boolean @@ Tags.Disjunction].zero

res27: Boolean @@ Tags.Disjunction = false

In [29]:
Monoid[Boolean @@ Tags.Conjunction].zero |+| Tags.Conjunction(true)

res28: Boolean @@ Tags.Conjunction = true

In [30]:
Monoid[Boolean @@ Tags.Conjunction].zero |+| Tags.Conjunction(false)

res29: Boolean @@ Tags.Conjunction = false

### Monoid としての Ordering

In [30]:
Ordering.LT |+| Ordering.GT

cmd30.sc:1: value |+| is not a member of object scalaz.Ordering.LT
val res30 = Ordering.LT |+| Ordering.GT
                        ^

: 

In [31]:
(Ordering.LT: Ordering) |+| (Ordering.GT: Ordering)

res30: Ordering = LT

In [32]:
(Ordering.GT: Ordering) |+| (Ordering.LT: Ordering)

res31: Ordering = GT

In [33]:
Monoid[Ordering].zero |+| (Ordering.LT: Ordering)

res32: Ordering = LT

In [34]:
Monoid[Ordering].zero |+| (Ordering.GT: Ordering)

res33: Ordering = GT

In [35]:
def lengthCompare(lhs: String, rhs: String): Ordering =
  (lhs.length ?|? rhs.length) |+| (lhs ?|? rhs)

defined function lengthCompare

In [36]:
lengthCompare("zen", "ants")

res35: Ordering = LT

In [37]:
lengthCompare("zen", "ant")

res36: Ordering = GT